In [ ]:
import requests
import bs4
import re
import urllib
from pathlib import Path
from IPython.display import Image
import random
import pandas as pd

In [ ]:
base_path = Path.cwd()
print(base_path)

In [ ]:
base_path = Path.cwd()
for year in range(2007, 2019):
    for sub_dir in range(0, 40000, 1000):
        data_dir = Path(base_path, str(year), str(sub_dir).zfill(6))
        data_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
HOME_PAGE = 'https://eaccess.dccourts.gov/eaccess/home.page'
session = requests.session()
r = session.get(HOME_PAGE)

assert r.ok
soup = bs4.BeautifulSoup(r.content, 'html.parser')
url = re.match(r'^.*url=(.*)(?= .*)?$', soup.find('meta').attrs['content']).groups()[0]

r = session.get(urllib.parse.urljoin(HOME_PAGE, url))
soup = bs4.BeautifulSoup(r.content, 'html.parser')

# Get the captcha
the_form = soup.findAll('form')[0]
the_form.findAll('input')
url = the_form.findAll('img', {'class': 'captchaImg'})[0].attrs['src']
img_url = urllib.parse.urljoin(r.url, url)
img_r = session.get(img_url)

assert img_r.ok

Image(img_r.content)

In [ ]:
captcha_content = input('captcha text: ')

In [ ]:
appendix = re.match(r"^.*'(\?x=.*?)'.*$", the_form.find('a', attrs={'name': 'linkFrag:beginButton'}).attrs['onclick']).groups()[0]
action_url = urllib.parse.urljoin(r.url, appendix)
the_form.attrs['method']

new_r = session.post(action_url, data={
    'captchaPanel:challengePassword': captcha_content,
    'idb_hf_0': None,
    'linkFrag:beginButton': 1 })

assert new_r.ok

In [ ]:
def scrape_case(new_r, year: int, case_num: int):
    sub_dir = str(int(round((case_num+499)/1000, 0))*1000).zfill(6)
    data_dir = Path(base_path, str(year), str(sub_dir).zfill(6))

    caseDscr = str(year) + ' CF2 ' + str(case_num).zfill(6)
    file_name = caseDscr + '.html'

    new_soup = bs4.BeautifulSoup(new_r.content, 'html.parser')

    parsed_url = urllib.parse.urlparse(new_r.url)
    d = urllib.parse.parse_qs(parsed_url.query)
    d['random'] = [str(random.random())]
    parsed_url = parsed_url._replace(query=urllib.parse.urlencode({k: v[0] for k, v in d.items()}))
    new_url = urllib.parse.urlunparse(parsed_url)
    new_r = session.get(new_url)

    new_form = new_soup.find('form')
    action = new_form.attrs['action']
    action_url = urllib.parse.urljoin(new_r.url, action)
    new_new_r = session.get(action_url, data={
        'id24_hf_0': None,
        'caseDscr': caseDscr,
        'submitLink': 'Search'
    }, headers={'referer': new_r.url}, allow_redirects=True)
    c = bs4.BeautifulSoup(new_new_r.content, 'html.parser')
    x_val = re.findall(r"wicketAjaxGet\('\?x=(.*?)'", c.find_all('script')[-3].text)[0]

    parsed_url = urllib.parse.urlparse(new_r.url)
    d = urllib.parse.parse_qs(parsed_url.query)
    d['x'] = [str(x_val)]
    d['random'] = [str(random.random())]
    parsed_url = parsed_url._replace(query=urllib.parse.urlencode({k: v[0] for k, v in d.items()}))
    new_url = urllib.parse.urlunparse(parsed_url)
    new_r = session.get(new_url)


    new_form = new_soup.find('form')
    action = new_form.attrs['action']
    action_url = urllib.parse.urljoin(new_r.url, action)
    new_new_r = session.post(action_url, data={
        'id24_hf_0': None,
        'caseDscr': caseDscr,
        'submitLink': 'Search'
    }, headers={'referer': new_r.url}, allow_redirects=True)

    soup = bs4.BeautifulSoup(new_new_r.content, 'html.parser')

    if soup.find("div", {"id": "srchResultNoticeNomatch"})==None:
        print(case_num, end = ' ')
        appendix = soup.find('table').find('tbody').find_all('tr')[0].find('a', attrs={'id': 'grid$row:1$cell:3$link'}).attrs['href']
        r = session.get(urllib.parse.urljoin(new_new_r.url, appendix))
        content = bs4.BeautifulSoup(r.content, 'html.parser').prettify()
        with open(Path(data_dir,file_name), 'w') as f:
          f.write(content)

In [ ]:
# This small section checks to see if there have been any developments in previously scraped cases that were undisposed. 
# It does not check to see if there is new information in the case, but it would be nice if it did! 
# As it stands now, it will likely re-scrape most of the cases in the most recent year. 
# Because the scraping of new cases starts with the most recently created file, this should run before new cases are scraped. 
# It might be that a recent case is re-scraped if it was already disposed, but that will be relatively rare. 
# There may be a better way with refactoring, but this should work, at least for now. 

for year in range(2014, 2020):
    file_name = 'dispositions_data_'+str(year)+'.csv'
    print("\n",file_name)
    try:
        dispositions = pd.read_csv(Path(base_path.parent,'DCSC-Parser',file_name))
        undisposed = dispositions.Disposition == "Undisposed"
        for row in dispositions[undisposed].itertuples():
            case_num = int(row.case_id[9:15])
            scrape_case(new_r, year=year, case_num=case_num)
    except:
        pass


In [ ]:
# when starting hot, this starts at the most recent undisposed case. 

year = 2019
num = 0
starting_hot = 1

if starting_hot == 1:
    folder_path = Path(base_path,str(year))
    print(folder_path.absolute())
    list_of_paths = folder_path.rglob('*.html') 
    latest_path = max(list_of_paths, key=lambda p: p.stat().st_ctime)
    start_year = int(latest_path.name[:4])
    start_num = int(latest_path.name[9:15])
else: 
    start_year, start_num = year, num

print(start_year, start_num)

for case_num in range(start_num, 30000):
    if(case_num/100 == round(case_num/100)): 
        print('  ')
    scrape_case(new_r, year, case_num)
    latest_scrape = {'year':year, 'case_number':case_num}
    case_id = str(year) + ' CF2 ' + str(case_num).zfill(6)

In [ ]:
print("Done!")